In [1]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv('Amazon.csv')
data.head()

,AKM1MP6P0OYPR,0132793040,5.0,1365811200
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600


In [5]:
data.columns = ['user_id', 'product_id','ratings','timestamp']

In [7]:
data.head()

,user_id,product_id,ratings,timestamp
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600


In [9]:
data.shape

(7824481, 4)

In [12]:
df = data[:int(len(data)*.1)]
df.shape

(782448, 4)

In [14]:
counts = df['user_id'].value_counts()
counts

A5JLAU2ARJ0BO     384
A231WM2Z2JL0U3    249
A25HBO5V8S8SEA    163
A6FIAB28IS79      113
AT6CZDCP4TRGA     112
                 ... 
AYK3XA6M9FGSC       1
AQ1OMXIN4TQ0H       1
A2M4KVAS2J9O3C      1
AAQBM3POZKT9H       1
AY5IAT0TP4OAW       1
Name: user_id, Length: 606149, dtype: int64

In [19]:
data1 = df[df['user_id'].isin(counts[counts>=50].index)]
data1.head()

,user_id,product_id,ratings,timestamp
2161,A5JLAU2ARJ0BO,1400532655,1.0,1291334400
7380,A2AEZQ3DGBBLPR,B000000O48,5.0,1038873600
7447,A2R6RA8FRBS608,B000001OL6,4.0,1209513600
7788,A11D1KHM7DVOQK,B000001OMN,2.0,1167350400
8731,A6FIAB28IS79,B00000J05A,3.0,985564800


In [21]:
data1.groupby('product_id')['ratings'].mean().sort_values(ascending=False)

product_id
B000068IGI    5.0
B00006IJO4    5.0
B00006JI6T    5.0
B0001CNKCM    5.0
B000816XI4    5.0
             ... 
B00005QWZ7    1.0
B00009W5QR    1.0
B00005RKO5    1.0
B00005UV2R    1.0
1400532655    1.0
Name: ratings, Length: 2207, dtype: float64

In [32]:
final_ratings = data.pivot(index = 'user_id', columns='product_id', values='ratings').fillna(0)
final_ratings.shape

(27, 2207)

In [36]:
number_of_ratings = np.count_nonzero(final_ratings)
possible_ratings = final_ratings.shape[0]*final_ratings.shape[1]
density = (number_of_ratings/possible_ratings)*100
final_ratings_T = final_ratings.transpose()

In [58]:
grouped = data.groupby('product_id').agg({'user_id':'count'}).reset_index()
grouped.rename(columns = {'user_id':'score'},inplace=True)
training_data = grouped.sort_values(['score','product_id'], ascending=[0,1])
training_data['Rank'] = training_data['score'].rank(ascending=0,method='first')
recommendations = training_data.head()
recommendations

,product_id,score,Rank
113,B00004SB92,6,1.0
1099,B00008OE6I,5,2.0
368,B00005AW1H,4,3.0
612,B0000645C9,4,4.0
976,B00007KDVI,4,5.0


In [70]:
def recommend(id):
    recommend_products = recommendations
    recommend_products['user_id'] = id
    column = recommend_products.columns.tolist()
    column = column[-1:]+column[:-1]
    reccommend_products = recommend_products[column]
    return recommend_products
print(recommend(3))

      product_id  score  Rank  user_id
113   B00004SB92      6   1.0        3
1099  B00008OE6I      5   2.0        3
368   B00005AW1H      4   3.0        3
612   B0000645C9      4   4.0        3
976   B00007KDVI      4   5.0        3


C:\Windows\Temp\ipykernel_3276\2676246591.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommend_products['user_id'] = id
